# Aggregating and Combining `pandas` DataFrames

## Objectives

- Use GroupBy objects to organize and aggregate data
- Create pivot tables from DataFrames
- Combine DataFrames by merging, joining, and concatinating

## Set Up

Surprise, surprise... we're still working with the Austin Animal Center Data! Let's start with Outcomes

In [87]:
# Imports

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [88]:
outcomes = pd.read_csv('data/Austin_Animal_Center_Outcomes_022822.csv',
                       parse_dates=['DateTime', 'Date of Birth'])

/var/folders/r7/dkcrrdg1099bt6wjv8tr4trc0000gn/T/ipykernel_16176/4130278359.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  outcomes = pd.read_csv('data/Austin_Animal_Center_Outcomes_022822.csv',


In [89]:
outcomes.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A794011,Chunk,2019-05-08 18:20:00,May 2019,2017-05-02,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
1,A776359,Gizmo,2018-07-18 16:02:00,Jul 2018,2017-07-12,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown
2,A821648,NaN,2020-08-16 11:38:00,Aug 2020,2019-08-16,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray
3,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
4,A674754,NaN,2014-03-18 11:47:00,Mar 2014,2014-03-12,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby


In [90]:
outcomes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137097 entries, 0 to 137096
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Animal ID         137097 non-null  object        
 1   Name              96095 non-null   object        
 2   DateTime          137097 non-null  datetime64[ns]
 3   MonthYear         137097 non-null  object        
 4   Date of Birth     137097 non-null  datetime64[ns]
 5   Outcome Type      137073 non-null  object        
 6   Outcome Subtype   62653 non-null   object        
 7   Animal Type       137097 non-null  object        
 8   Sex upon Outcome  137095 non-null  object        
 9   Age upon Outcome  137092 non-null  object        
 10  Breed             137097 non-null  object        
 11  Color             137097 non-null  object        
dtypes: datetime64[ns](2), object(10)
memory usage: 12.6+ MB


In [91]:
# Let's create our Age in Days column
outcomes['Calculated Age in Days'] = pd.to_datetime(outcomes['DateTime'].dt.date) - outcomes['Date of Birth']

# dt.date tells you just to grab the date and not the time

In [92]:
# Grab just the integer here...
outcomes['Calculated Age in Days'] = outcomes['Calculated Age in Days'].dt.days

In [93]:
# Sanity check
outcomes.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color,Calculated Age in Days
0,A794011,Chunk,2019-05-08 18:20:00,May 2019,2017-05-02,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White,736
1,A776359,Gizmo,2018-07-18 16:02:00,Jul 2018,2017-07-12,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown,371
2,A821648,NaN,2020-08-16 11:38:00,Aug 2020,2019-08-16,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray,366
3,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff,128
4,A674754,NaN,2014-03-18 11:47:00,Mar 2014,2014-03-12,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby,6


## Aggregating over DataFrames: `.groupby()`

Those of you familiar with SQL have probably used the GROUP BY command. (And if you haven't, you'll see it very soon!) Pandas has this, too.

The `.groupby()` method is especially useful for aggregate functions applied to the data grouped in particular ways.

In [94]:
outcomes.describe()

,DateTime,Date of Birth,Calculated Age in Days
count,137097,137097,137097.000000
mean,2017-08-14 19:16:10.551215872,2015-06-11 04:44:47.533644032,794.991575
min,2013-10-01 09:31:00,1991-09-22 00:00:00,-1116.000000
25%,2015-08-07 17:37:00,2013-08-10 00:00:00,94.000000
50%,2017-07-09 14:55:00,2015-10-01 00:00:00,370.000000
75%,2019-06-28 10:48:00,2018-03-07 00:00:00,914.000000
max,2022-02-28 14:19:00,2022-02-18 00:00:00,10996.000000
std,NaN,NaN,1069.193353


In [95]:
# Just using groupby outputs some weird GroupBy object... not helpful
outcomes.groupby('Animal Type').mean(numeric_only=True)

# this allows you to perform a method on a specific group w/i the dataset

# it looks like dogs are older than other animals, but there might be outliars influencing this
# so we need to check the data

,Calculated Age in Days
Animal Type,
Bird,533.716981
Cat,532.816939
Dog,1004.114514
Livestock,503.800000
Other,479.143665


Once we know we are working with a type of object, it opens up a suite of attributes and methods. One attribute we can look at is `groups`.

In [96]:
# This returns each group indexed by the group name, e.g. 'Bird',
# along with the row indices of each value

outcomes.groupby('Animal Type').groups

{'Bird': [206, 534, 985, 1027, 1284, 1310, 2220, 2258, 2274, 2417, 2521, 2598, 2712, 2778, 3178, 3379, 3648, 3743, 4003, 4024, 4288, 4702, 4766, 4998, 5063, 5205, 5436, 5656, 5848, 6087, 6236, 6340, 6592, 6682, 7033, 7352, 7428, 7985, 8048, 8315, 8331, 8414, 8538, 8922, 9203, 9448, 9758, 9825, 10103, 10165, 10407, 10657, 10736, 11386, 11616, 11674, 11732, 11765, 11771, 11837, 12205, 12418, 12423, 12451, 12474, 12713, 12902, 12978, 13057, 13063, 13095, 13272, 13317, 13323, 13435, 13474, 13677, 13934, 13950, 13963, 13981, 14108, 14131, 14146, 14193, 15114, 15193, 15543, 15553, 15813, 16022, 16197, 16499, 16866, 17173, 17338, 17390, 17426, 18319, 18359, ...], 'Cat': [0, 4, 7, 8, 10, 11, 14, 15, 16, 17, 18, 20, 24, 26, 34, 37, 49, 54, 56, 66, 67, 68, 70, 75, 78, 80, 83, 84, 89, 90, 92, 94, 95, 97, 98, 102, 113, 115, 116, 117, 118, 120, 122, 126, 139, 141, 142, 145, 147, 148, 151, 152, 156, 157, 158, 164, 167, 168, 170, 171, 176, 178, 184, 191, 192, 194, 200, 202, 203, 207, 209, 212, 215, 2

In [97]:
# Same goes for multi-index groupbys
animal_outcome = outcomes.groupby(['Animal Type', 'Outcome Type'])

In [98]:
# .groups outputs a dictionary, so we can access the group names using keys()
animal_outcome.groups.keys()

dict_keys([('Bird', 'Adoption'), ('Bird', 'Died'), ('Bird', 'Disposal'), ('Bird', 'Euthanasia'), ('Bird', 'Missing'), ('Bird', 'Relocate'), ('Bird', 'Return to Owner'), ('Bird', 'Transfer'), ('Cat', 'Adoption'), ('Cat', 'Died'), ('Cat', 'Disposal'), ('Cat', 'Euthanasia'), ('Cat', 'Missing'), ('Cat', 'Relocate'), ('Cat', 'Return to Owner'), ('Cat', 'Rto-Adopt'), ('Cat', 'Transfer'), ('Cat', nan), ('Dog', 'Adoption'), ('Dog', 'Died'), ('Dog', 'Disposal'), ('Dog', 'Euthanasia'), ('Dog', 'Missing'), ('Dog', 'Return to Owner'), ('Dog', 'Rto-Adopt'), ('Dog', 'Transfer'), ('Dog', nan), ('Livestock', 'Adoption'), ('Livestock', 'Died'), ('Livestock', 'Euthanasia'), ('Livestock', 'Return to Owner'), ('Livestock', 'Transfer'), ('Other', 'Adoption'), ('Other', 'Died'), ('Other', 'Disposal'), ('Other', 'Euthanasia'), ('Other', 'Missing'), ('Other', 'Relocate'), ('Other', 'Return to Owner'), ('Other', 'Rto-Adopt'), ('Other', 'Transfer'), ('Other', nan)])

In [99]:
# We can then get a specific group, such as cats that were adopted
animal_outcome.get_group(('Cat', 'Adoption'))

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color,Calculated Age in Days
7,A689724,*Donatello,2014-10-18 18:52:00,Oct 2014,2014-08-01,Adoption,NaN,Cat,Neutered Male,2 months,Domestic Shorthair Mix,Black,78
8,A680969,*Zeus,2014-08-05 16:59:00,Aug 2014,2014-06-03,Adoption,NaN,Cat,Neutered Male,2 months,Domestic Shorthair Mix,White/Orange Tabby,63
20,A730621,*Liza,2016-09-10 18:59:00,Sep 2016,2016-05-18,Adoption,NaN,Cat,Spayed Female,3 months,Domestic Shorthair Mix,Calico,115
26,A801106,NaN,2019-08-16 14:05:00,Aug 2019,2019-05-06,Adoption,NaN,Cat,Neutered Male,3 months,Domestic Shorthair,Orange Tabby,102
54,A792258,Vesper,2019-04-10 20:53:00,Apr 2019,2016-09-08,Adoption,NaN,Cat,Spayed Female,2 years,Domestic Shorthair Mix,Tortie,944
...,...,...,...,...,...,...,...,...,...,...,...,...,...
137072,A846689,Coco Chanel,2022-02-26 17:23:00,Feb 2022,2021-08-19,Adoption,NaN,Cat,Spayed Female,6 months,Domestic Shorthair,Blue Tabby,191
137073,A845330,Mitzi,2022-02-26 18:09:00,Feb 2022,2021-01-28,Adoption,NaN,Cat,Spayed Female,1 year,Domestic Shorthair,Torbie/White,394
137088,A851184,*Papaya,2022-02-28 11:38:00,Feb 2022,2021-02-08,Adoption,NaN,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Orange Tabby/White,385
137090,A847804,*Mahalia,2022-02-28 11:42:00,Feb 2022,2011-12-08,Adoption,NaN,Cat,Spayed Female,10 years,Domestic Shorthair Mix,Brown Tabby/White,3735


## Aggregating

Once again, as we will see in SQL, groupby objects are intended to be used with aggregation. In SQL, we will see that our queries that include GROUP BY require aggregation performed on columns.

We can use `.sum()`, `.mean()`, `.count()`, `.max()`, `.min()`, etc. Find a list of common aggregations [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html).

In [100]:
outcomes.groupby('Animal Type').max(numeric_only=True)

,Calculated Age in Days
Animal Type,
Bird,10996
Cat,8036
Dog,8766
Livestock,1844
Other,7671


## Exercise

Use `.groupby()` to find the most recent birth date of each (main) animal type.


In [101]:
# Your code here

<details>
    <summary>Answer</summary>

```python
outcomes.groupby('Animal Type')['Date of Birth'].max()
```
</details>

# Pivoting a DataFrame

## `.pivot_table()`

Those of you familiar with Excel have probably used Pivot Tables. Pandas has a similar functionality.

Grouping by two different columns can be very helpful.

In [102]:
outcomes.groupby(by=['Outcome Type', 'Sex upon Outcome']).agg('mean')

TypeError: agg function failed [how->mean,dtype->object]

But it has the unsavory side effect of creating a two-level index. This can be a good time to use `.pivot_table()`.

(There is also a `.pivot()`. For the somewhat subtle differences, see [here](https://stackoverflow.com/questions/30960338/pandas-difference-between-pivot-and-pivot-table-why-is-only-pivot-table-workin).)

In [ ]:
# Check it out!
outcomes.pivot_table(index='Outcome Type', columns=['Sex upon Outcome', 'Animal Type'], aggfunc='mean')

# Methods for Combining DataFrames: `.join()`, `.merge()`, `.concat()`

Many ways to combine dataframes! Luckily, pandas has great docs: https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## `.join()`

In [ ]:
toy1 = pd.DataFrame([[63, 142], [33, 47]], columns=['age', 'HP'])
toy2 = pd.DataFrame([[63, 100], [33, 200]], columns=['age', 'MP'])

toy1

In [ ]:
toy2

In [ ]:
# We can't just join these as they are, since we haven't specified our suffixes

toy1.join(toy2)

In [ ]:
toy1.join(toy2, lsuffix='1', rsuffix='2') # left and right suffix
# this is not very practical in the real world

If we don't want to keep both, we could set the overlapping column as the index in each DataFrame:

In [ ]:
toy1.set_index('age').join(toy2.set_index('age')) # we set the overlapping column as the index
# set_index allows us to use a column as the index --> important for time series

In [ ]:
toy1.drop('age', axis=1).join(toy2)

# drop 'age' from the first dataset
# axis = 1 means to look at a column instead of a row (row would be 0)

## `.merge()`

Or we could use `.merge()`:

In [ ]:
toy1.merge(toy2)

In [ ]:
ds_chars = pd.read_csv('data/ds_chars.csv', index_col=0)
ds_chars

In [ ]:
states = pd.read_csv('data/states.csv', index_col=0)
states

#this data set and the one above both have columns referring to state

## The `how` Parameter

This parameter in both `.join()` and `.merge()` tells the compiler what sort of join to effect. We'll cover this in detail when we discuss SQL.

![image showcasing how the how parameter in a join/merge would combine the two datasets, using venn-style diagrams](https://www.datasciencemadesimple.com/wp-content/uploads/2017/09/join-or-merge-in-python-pandas-1.png)
[[Image Source]](https://www.datasciencemadesimple.com/join-merge-data-frames-pandas-python/)

In [ ]:
ds_chars.merge(states,
               left_on='home_state',
               right_on='state',
               how='inner')

# left is outside (), right is inside ()
# you don't really ever  need to do an outer join

In [ ]:
ds_chars.merge(states,
               left_on='home_state',
               right_on='state',
               how='outer')

# this returns everything, whether or not there is a match

## `pd.concat()`

This method takes a *list* of pandas objects as arguments.

In [ ]:
prefs = pd.read_csv('data/preferences.csv', index_col=0)
prefs

# concat stacks data ontop of each other
# only useful if you have data files with the SAME EXACT columns
# Ex: Q1, Q2, Q3, and Q4 sets with the same exact columns, but you 
# need them all in one list for the year

In [ ]:
ds_full = pd.concat([ds_chars, prefs])
ds_full

# this didn't work bc they didn't have the same columns

`pd.concat()`–– and many other pandas operations –– make use of an `axis` parameter. For this particular method I need to specify whether I want to concatenate the DataFrames *row-wise* (`axis=0`) or *column-wise* (`axis=1`). The default is `axis=0`, so let's override that!

In [ ]:
ds_full = pd.concat([ds_chars, prefs], axis=1)
ds_full

## Back to the Center

We have Intakes data and we have Outcomes data... time to merge!

In [104]:
# Peek at the outcomes data we already had in here
outcomes.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color,Calculated Age in Days
0,A794011,Chunk,2019-05-08 18:20:00,May 2019,2017-05-02,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White,736
1,A776359,Gizmo,2018-07-18 16:02:00,Jul 2018,2017-07-12,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown,371
2,A821648,NaN,2020-08-16 11:38:00,Aug 2020,2019-08-16,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray,366
3,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff,128
4,A674754,NaN,2014-03-18 11:47:00,Mar 2014,2014-03-12,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby,6


In [103]:
# Read in the intakes data
intakes = pd.read_csv("data/Austin_Animal_Center_Intakes_022822.csv",
                      parse_dates=['DateTime'])
# Check it out
intakes.head()

/var/folders/r7/dkcrrdg1099bt6wjv8tr4trc0000gn/T/ipykernel_16176/2671916070.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  intakes = pd.read_csv("data/Austin_Animal_Center_Intakes_022822.csv",


,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
0,A786884,*Brock,2019-01-03 16:19:00,January 2019,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor
1,A706918,Belle,2015-07-05 12:59:00,July 2015,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver
2,A724273,Runster,2016-04-14 18:43:00,April 2016,2818 Palomino Trail in Austin (TX),Stray,Normal,Dog,Intact Male,11 months,Basenji Mix,Sable/White
3,A665644,NaN,2013-10-21 07:59:00,October 2013,Austin (TX),Stray,Sick,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico
4,A682524,Rio,2014-06-29 10:38:00,June 2014,800 Grove Blvd in Austin (TX),Stray,Normal,Dog,Neutered Male,4 years,Doberman Pinsch/Australian Cattle Dog,Tan/Gray


In [108]:
# Let's try merging on Animal ID
combined = outcomes.merge(intakes,
                          on='Animal ID',
                          how='inner',
                          suffixes=['_outcome', '_intake'])


# it's easy to merge on Animal Id since they are all unique
# we don't need a L or R since the column name is the same
# the suffix tells you which dataframe each column is from since some overlap like date/time

In [39]:
# What was the result?
combined.head()

,Animal ID,Name_outcome,DateTime_outcome,MonthYear_outcome,Date of Birth,Outcome Type,Outcome Subtype,Animal Type_outcome,Sex upon Outcome,Age upon Outcome,...,DateTime_intake,MonthYear_intake,Found Location,Intake Type,Intake Condition,Animal Type_intake,Sex upon Intake,Age upon Intake,Breed_intake,Color_intake
0,A794011,Chunk,2019-05-08 18:20:00,May 2019,2017-05-02,Rto-Adopt,NaN,Cat,Neutered Male,2 years,...,2019-05-02 16:51:00,May 2019,Austin (TX),Owner Surrender,Normal,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
1,A776359,Gizmo,2018-07-18 16:02:00,Jul 2018,2017-07-12,Adoption,NaN,Dog,Neutered Male,1 year,...,2018-07-12 12:46:00,July 2018,7201 Levander Loop in Austin (TX),Stray,Normal,Dog,Intact Male,1 year,Chihuahua Shorthair Mix,White/Brown
2,A821648,NaN,2020-08-16 11:38:00,Aug 2020,2019-08-16,Euthanasia,NaN,Other,Unknown,1 year,...,2020-08-16 10:10:00,August 2020,Armadillo Rd And Clubway Ln in Austin (TX),Wildlife,Sick,Other,Unknown,1 year,Raccoon,Gray
3,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,...,2016-02-08 11:05:00,February 2016,Dove Dr And E Stassney in Austin (TX),Stray,Normal,Dog,Intact Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
4,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,...,2016-02-15 10:37:00,February 2016,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff


In [109]:
combined.shape

(176664, 24)

In [110]:
intakes.shape

(136763, 12)

In [114]:
outcomes.shape

# you should never have more observations in your inner join than in your individual data frames
# your combined data has more observations than the intakes or outcomes individually
# take note of duplicates in your data

(137097, 13)

Let's discuss/explore: did that work the way we expected?

- 

<details>
    <summary>Observation Notes</summary>

- We went from about 136k rows in each of the dataframes to 176k! Even using an inner join! Something seems off. 
    
    
</details>

In [117]:
# We might want to try something different
# Can we clean something to make a better merge?
combined.loc[combined.duplicated(subset='Animal ID')]

# this tells us we have over 5000 duplicates

# always check your merge

,Animal ID,Name_outcome,DateTime_outcome,MonthYear_outcome,Date of Birth,Outcome Type,Outcome Subtype,Animal Type_outcome,Sex upon Outcome,Age upon Outcome,...,DateTime_intake,MonthYear_intake,Found Location,Intake Type,Intake Condition,Animal Type_intake,Sex upon Intake,Age upon Intake,Breed_intake,Color_intake
4,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,...,2016-02-15 10:37:00,February 2016,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
7,A659412,Princess,2020-10-05 14:37:00,Oct 2020,2013-03-24,Adoption,NaN,Dog,Spayed Female,7 years,...,2020-10-05 11:20:00,October 2020,Austin (TX),Owner Surrender,Normal,Dog,Spayed Female,7 years,Chihuahua Shorthair Mix,Brown
15,A818049,Fiona,2020-06-01 13:24:00,Jun 2020,2018-06-01,Return to Owner,NaN,Dog,Intact Female,2 years,...,2020-09-26 18:37:00,September 2020,Cambourne Dr & Hariss Ridge Blvd in Austin (TX),Stray,Normal,Dog,Intact Female,2 years,Pit Bull,White/Blue
16,A818049,Fiona,2020-06-01 13:24:00,Jun 2020,2018-06-01,Return to Owner,NaN,Dog,Intact Female,2 years,...,2021-07-12 09:18:00,July 2021,Harris Ridge Boulevard in Pflugerville (TX),Stray,Normal,Dog,Intact Female,3 years,Pit Bull,White/Blue
21,A698049,Luigi,2015-03-16 14:50:00,Mar 2015,2014-06-05,Transfer,Partner,Cat,Spayed Female,9 months,...,2015-03-05 13:01:00,March 2015,Austin (TX),Owner Surrender,Normal,Cat,Spayed Female,8 months,Domestic Medium Hair Mix,Black/White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176651,A690359,*Porridge,2022-02-26 11:34:00,Feb 2022,2014-06-19,Transfer,Partner,Dog,Neutered Male,7 years,...,2021-02-21 14:57:00,February 2021,2603 Canterbury Street in Austin (TX),Owner Surrender,Behavior,Dog,Neutered Male,6 years,Labrador Retriever Mix,Tan/White
176652,A690359,*Porridge,2022-02-26 11:34:00,Feb 2022,2014-06-19,Transfer,Partner,Dog,Neutered Male,7 years,...,2014-10-19 11:35:00,October 2014,Robert Martinez And 4Th in Austin (TX),Stray,Normal,Dog,Intact Male,4 months,Labrador Retriever Mix,Tan/White
176655,A850725,Monty,2022-02-25 11:23:00,Feb 2022,2020-01-30,Adoption,NaN,Dog,Neutered Male,2 years,...,2022-02-28 11:05:00,February 2022,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,2 years,Labrador Retriever,Black/White
176662,A813933,Lucille,2022-02-28 14:19:00,Feb 2022,2018-12-21,Adoption,NaN,Dog,Spayed Female,3 years,...,2020-02-21 10:00:00,February 2020,11402 February Drive in Austin (TX),Stray,Normal,Dog,Spayed Female,1 year,Belgian Malinois,Brown/Black


In [118]:
# Try again
clean_intakes = intakes.drop_duplicates(subset=['Animal ID'])
clean_outcomes = outcomes.drop_duplicates(subset=['Animal ID'])

In [119]:
clean_combined_df = clean_intakes.merge(clean_outcomes, on='Animal ID',
                                        how='inner',
                                        suffixes=['_intake', '_outcome'])

In [120]:
clean_combined_df.shape

(121773, 24)

# Level Up: Quick Column Name Clean Up Code

Throwing a quick use of a lambda function your way:

In [ ]:
outcomes_renamed = outcomes.rename(columns = lambda x: x.replace(" ", "_").lower())
outcomes_renamed.head()

# Level Up: `pandas.set_option()`

We can adjust how `pandas` works by setting options in advance.

For complete documentation, see [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html).

## Block Scientific Notation

For example, suppose we want to prevent numbers from being displayed in scientific notation.

In [ ]:
df = pd.DataFrame([[1e9, 2e9], [3e9, 4e9]])
df

Then we can use:

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)

df

## See More Rows

Or suppose we want `pandas` to show more rows.

In [ ]:
df2 = pd.DataFrame(np.array(range(100)))
df2

In that case we can use:

In [ ]:
pd.set_option('display.max_rows', 100)

df2